In [1]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd
import os
import dotenv

dotenv.load_dotenv(override=True)

False

In [2]:
OUTPUT_DIR = "../../gemini-benchmark/outputs/mawpsmultiarith"

In [3]:
models = os.listdir(OUTPUT_DIR)

In [4]:
# Upload base dataset
df = pd.read_json(os.path.join(OUTPUT_DIR, models[0], "output.jsonl"), lines=True)
base_df = pd.DataFrame({
    "qid": df["qid"],
    "question": df["question"],
    "answer": df["answer"].astype(str) 
})

In [5]:
zeno_client = ZenoClient("zen_P4CK880bWHV2dJLbAenb0r8Gf6QNLdTXhSkkHqPDm4I")

/data/tir/projects/tir4/users/sakter/anaconda3/envs/bleed/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package zeno-client is out of date. Your version is 0.1.13, the latest is 0.1.14.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [6]:
project = zeno_client.create_project(
    name="Gemini Evaluation - MawpsMultiArith",
    description="Evaluation of Gemini, GPT-4, and Mixtral on MawpsMultiArith dataset",
    view={
        "data": {
            "type": "text"
        },
        "label": {
            "type": "text"
        },
        "output": {
            "type": "markdown"
        }
    },
    public=True,
    metrics=[
        ZenoMetric(name="Accuracy Strict Match", type="mean", columns=["is_correct"]),
        ZenoMetric(name="Accuracy", type="mean", columns=["is_correct_last"])
    ],
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/16eda8f8-7edb-4241-9095-8b71b3a2507f/Gemini%20Evaluation%20-%20MawpsMultiArith


In [7]:
project.upload_dataset(base_df, id_column="qid", data_column="question", label_column="answer")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded data


In [10]:
def check_fraction(num):
    if num % 1 != 0:
        return 1
    return 0

for model in models:
    df = pd.read_json(os.path.join(OUTPUT_DIR, model, "output.jsonl"), lines=True)
    if model == 'mixtral':
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['generated_text'].split('Q:')[0]}\n\n**{x['predict']}**", axis=1),
            "fraction": df.apply(lambda x: check_fraction(float(x['answer'])), axis=1),
            "numeric_answer": df.apply(lambda x: float(x['answer']), axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['generated_text'].split('Q:')[0].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool)
        })
    else:
        output_df = pd.DataFrame({
            "qid": df["qid"],
            "output": df.apply(lambda x: f"{x['generated_text'].split('Q:')[0]}\n\n**{x['predict']}**", axis=1),
            "fraction": df.apply(lambda x: check_fraction(float(x['answer'])), axis=1),
            "numeric_answer": df.apply(lambda x: float(x['answer']), axis=1),
            "question_length": df.apply(lambda x: len(x['question'].split(' ')), axis=1),
            "output_length": df.apply(lambda x: len(x['generated_text'].split(' ')), axis=1),
            "is_correct": df["is_correct"].astype(bool),
            "is_correct_last": df["is_correct_last"].astype(bool)
        })
    if model == 'gpt-4-1106-preview':
        model = 'gpt-4-turbo'
    project.upload_system(output_df, name=model, id_column="qid", output_column="output")

  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system


  0%|          | 0/1 [00:00<?, ?it/s]

Successfully uploaded system
